In [1]:
# !pip3 install transformers
# !pip install datasets
# !pip3 install emoji

In [2]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import transformers
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from transformers import get_linear_schedule_with_warmup, AdamW, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset

In [3]:
manual_seed = 2022
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

## BERTweet

### 1. bertweet-base model

BERTweet: A pre-trained language model for English Tweets
BERTweet is the first public large-scale language model pre-trained for English Tweets. BERTweet is trained based on the RoBERTa pre-training procedure. The corpus used to pre-train BERTweet consists of 850M English Tweets (16B word tokens ~ 80GB), containing 845M Tweets streamed from 01/2012 to 08/2019 and 5M Tweets related to the COVID-19 pandemic. The general architecture and experimental results of BERTweet can be found in our paper: https://huggingface.co/vinai/bertweet-base

Cited from https://github.com/VinAIResearch/BERTweet 

"Normalize raw input tweets

Before applying BPE to the pre-training corpus of English Tweets, we tokenized these Tweets using TweetTokenizer from the NLTK toolkit and used **the emoji package to translate emotion icons into text strings** (here, each icon is referred to as a word token). 

We also normalized the Tweets by **converting user mentions and web/url links into special tokens @USER and HTTPURL**, respectively. Thus it is recommended to also apply the same pre-processing step for BERTweet-based downstream applications w.r.t. the raw input Tweets.

BERTweet provides this pre-processing step by enabling the normalization argument of its tokenizer. `tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)`"


## Data Preprocessing

We are using [ucberkeley-dlab_measuring-hate-speech](https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech) as our dataset. To learn more about it, go to notebooks/data_description.ipynb


_In our dataset, the labels are organized like this:_
- 0 : no
- 1 : unclear
- 2 : yes

In [4]:
import datasets 

dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')["train"]
data = dataset.to_pandas()[["text", "hatespeech"]]

Using custom data configuration ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd
Reusing dataset parquet (/Users/oksanakurylo/.cache/huggingface/datasets/parquet/ucberkeley-dlab--measuring-hate-speech-7cb9b0b8e4d0e1dd/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/1 [00:00<?, ?it/s]

1. Delete label - 1  (unclear) and keep only:
0 - no,
2 - yes
2. Change label 2 to 1, so to have two labels: 
0 - no,
1 - yes

In [5]:
hatespeech_unk = data[data["hatespeech"] == 1].index
data.drop(hatespeech_unk , inplace=True)
data = data.rename(columns={"hatespeech": "label"})
data.loc[data["label"] == 2.0, "label"] = 1

In [6]:
data["label"].value_counts()

0.0    80624
1.0    46021
Name: label, dtype: int64

In [7]:
train_data, temp_data = train_test_split(data, test_size=0.2)
dev_data, test_data = train_test_split(temp_data, test_size=0.5)

In [8]:
print(f"Train data size: {len(train_data)}")
print(f"Test data size: {len(test_data)}")
print(f"Dev data size: {len(dev_data)}")

Train data size: 101316
Test data size: 12665
Dev data size: 12664


## Performance Metrics

We are using precision, recall, and F1-score to measure the performance of the model.

In [9]:
def calculate_metrics(preds, labels):
    results = dict()
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    results['precision_score'] = precision_score(labels_flat, preds_flat, average='binary')
    results['recall_score'] = recall_score(labels_flat, preds_flat, average='binary')
    results['f1_score'] = f1_score(labels_flat, preds_flat, average='binary')
    return results

## Loading Tokenizer and encoding data

In [10]:
def encode_data(df, tokenizer):
    input_ids = []
    attention_masks = []
    for tweet in df[["text"]].values:
        tweet = tweet.item()
        encoded_data = tokenizer.encode_plus(
                            tweet,                      
                            add_special_tokens = True,  
                            max_length = 128,
                            padding = 'max_length',
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
        input_ids.append(encoded_data['input_ids'])
        attention_masks.append(encoded_data['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_ids': input_ids,
    'input_mask': attention_masks}
    return inputs

## Prepare DataLoaders

In [11]:
MODEL = "vinai/bertweet-base"
batch_size = 32

def prepare_dataloaders(train_df, dev_df, test_df, model_name, batch_size):
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, normalization=True)

    data_train = encode_data(train_df, tokenizer)
    labels_train = train_df.label.astype(int)

    data_valid = encode_data(dev_df, tokenizer)
    labels_valid = dev_df.label.astype(int)

    data_test = encode_data(test_df, tokenizer)

    input_ids, attention_masks = data_train.values()
    train_labels = torch.tensor(labels_train.values)
    train_dataset = TensorDataset(input_ids, attention_masks, train_labels)

    input_ids, attention_masks = data_valid.values()
    valid_labels = torch.tensor(labels_valid.values)
    val_dataset = TensorDataset(input_ids, attention_masks, valid_labels)

    input_ids, attention_masks = data_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )

    validation_dataloader = DataLoader(
                val_dataset, 
                sampler = SequentialSampler(val_dataset),
                batch_size = batch_size 
            )

    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )

    return train_dataloader, validation_dataloader, test_dataloader

In [12]:
train_dataloader, validation_dataloader, test_dataloader = prepare_dataloaders(train_data, dev_data, test_data, model_name=MODEL, batch_size = batch_size)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Setting up Pretrained BERTweet Model

In [13]:
def prepare_model(total_labels, model_name, model_to_load=None):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels = total_labels,  
        output_attentions = False, 
        output_hidden_states = False,
    )
    if model_to_load is not None:
        try:
            model.roberta.load_state_dict(torch.load(model_to_load))
            print("Loaded pre-trained model")
        except:
            pass
    return model

## Setting up Optimizer and Scheduler

In [14]:
def prepare_optimizer_scheduler(total_steps, learning_rate=5e-5):
    optimizer = AdamW(model.parameters(),
                    lr = learning_rate,
                    eps = 1e-8,
                    weight_decay = 1e-2
                    )
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps
                                                )
    return optimizer, scheduler

## Creating our Training Loop

In [15]:
EPOCHS = 3
NUM_LABELS = len(data.label.unique())
TOTAL_STEPS = len(train_dataloader) * EPOCHS
model = prepare_model(total_labels=NUM_LABELS, model_name=MODEL, model_to_load=None)
model.to(device)
optimizer, scheduler = prepare_optimizer_scheduler(total_steps=TOTAL_STEPS, learning_rate=5e-5)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [16]:
def evaluate(model, validation_dataloader):
    
    model.eval()
    preds = []
    true_labels = []
    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in tqdm(validation_dataloader):    
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits

        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
            
        preds.append(logits)
        true_labels.append(label_ids)
        
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    tqdm.write(f"Avg validation loss: {avg_val_loss}")

    return preds, true_labels, avg_val_loss

In [17]:
def train(model, optimizer, scheduler, train_dataloader, validation_dataloader, epochs):
    training_stats = []
    model.train() 
    total_train_loss = 0
    
    for epoch in tqdm(range(1, epochs+1)):
        progress_bar = tqdm(train_dataloader, 
                        desc=" Epoch {:1d}".format(epoch),
                        leave=False, # to overwrite each epoch
                        disable=False)
        
        for batch in progress_bar:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            model.zero_grad() 
            outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
            
            loss = outputs.loss
            logits = outputs.logits
            
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
        avg_train_loss = total_train_loss / len(train_dataloader) 
        tqdm.write(f"\nEpoch: {epoch}")
        tqdm.write(f"Training loss: {avg_train_loss}")
        
        preds, val_labels, avg_val_loss = evaluate(model, validation_dataloader)
        predictions = np.concatenate(preds, axis=0)
        labels = np.concatenate(val_labels, axis=0)
        
        scores = calculate_metrics(predictions, labels)
        precision = scores['precision_score']*100
        recall = scores['recall_score']*100
        f1 = scores['f1_score']*100
        tqdm.write(f"Precision Score: {precision}")
        tqdm.write(f"Recall Score: {recall}")
        tqdm.write(f"F1 Score: {f1}")
    print("Training complete!")

In [31]:
train(model, optimizer, scheduler, train_dataloader, validation_dataloader, EPOCHS)

  0%|          | 0/3 [20:53<?, ?it/s]


Epoch: 1
Training loss: 0.3227876849921863



 33%|███▎      | 1/3 [21:46<43:33, 1306.87s/it]

Avg validation loss: 0.34877841131328935
Precision Score: 81.57657657657657
Recall Score: 77.82552642887839
F1 Score: 79.65691664833956



 33%|███▎      | 1/3 [42:22<43:33, 1306.87s/it]

...

Avg validation loss: 0.358727316165136
Precision Score: 78.29778422033547
Recall Score: 81.24194241512677
F1 Score: 79.74269745861015



 67%|██████▋   | 2/3 [1:03:52<21:36, 1296.22s/it]


Epoch: 3
Training loss: 0.8674263991980117



100%|██████████| 3/3 [1:04:45<00:00, 1295.07s/it]

Avg validation loss: 0.358727316165136
Precision Score: 78.29778422033547
Recall Score: 81.24194241512677
F1 Score: 79.74269745861015
Training complete!
